### Installing required dependencies 

In [1]:
# You might need to re rerun this cell, if you see killed  at end of outcome of this cell
!pip install --upgrade pip
!pip install pandas
!pip install bioinfokit
!pip install goatools

     |████████████████████████████████| 2.0 MB 28.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.1.1
    Uninstalling pip-21.1.1:
      Successfully uninstalled pip-21.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 52.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 72.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 47.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 112.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 

  Created wheel for goatools: filename=goatools-1.2.3-py3-none-any.whl size=15764707 sha256=1fe30959b7e8a039ac00a29dd7dc9c5f88e400a6acf80dfaca593eaea4309659
  Stored in directory: /home/jovyan/.cache/pip/wheels/0b/59/dd/635dfe772e3359be7c53a00442da0959b7d5d76c75c928d560
Successfully built goatools
Killed


### Importing dependencies

In [2]:
%matplotlib inline
import pandas as pd
from pylab import *
from bioinfokit import analys, visuz
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import seaborn as sns
import textwrap
from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
from simcore_sdk import node_ports

### Loading files

- The first file consist Gene, P.Value, Log FC columns	
- The second input file was download from [NCBI Gene](https://www.ncbi.nlm.nih.gov/gene/?term=%229606%22[Taxonomy+ID]+AND+alive[property]+AND+genetype+protein+coding[Properties]) and uploaded here

In [3]:
PORTS = await node_ports.ports()

In [8]:
input_1 = await(await PORTS.inputs)[0].get()
input_2 = await(await PORTS.inputs)[1].get()


downloading /tmp/simcorefiles/outFile/gene_result_mm.txt [5802869 bytes]: 100%|██████████| 5.80M/5.80M [00:00<00:00, 43.2Mbyte/s]


In [5]:
output_1 = await(await PORTS.outputs)[0].get()
output_2 = await(await PORTS.outputs)[1].get()

### Conditions

In [6]:
rcParams["figure.figsize"] = 20,16
pvalue = 0.05
logFC = 0
# logFC = np.abs(logFC)


### Additional variable

In [7]:



data1 = pd.read_csv(input_1)

# data1["log10(PValue)"] = -1*np.log10(data1["P.Value"])
data1["expression"] = "Not differentially expressed"
data1.loc[((data1["P.Value"]<pvalue) &(data1["Log FC"]>logFC)), "expression"]= "Upregulated"
data1.loc[((data1["P.Value"]<pvalue) &(data1["Log FC"]< -1*logFC)), "expression"]= "Downregulated"
data1[["BP", "CC", "MF"]] = ""
data1

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:
?visuz.GeneExpression.volcano

### Volcano plot

In [ ]:
visuz.GeneExpression.volcano(df=data1, 
                             lfc="Log FC",
                             lfc_thr=(logFC,logFC),# For left and right
                             pv="P.Value", 
                             show=True,
                             gfont=18,
                             axtickfontsize=18,
                             axlabelfontsize=18,
                             dim=(20, 16),
                             pv_thr=(pvalue,pvalue), # For left and right
                             geneid='Gene', 
                             figname='volcano',
                             genenames=("DNAH2", "SIGIRR"), # Just random selected genes Can be cnaged for significat genes
                             plotlegend=True)

### Downloading Gene2go

Note: wget must be in system path

In [ ]:

!wget -c ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz

ncbi_gene_results_to_python.py is part of [goatools](https://github.com/tanghaibao/goatools/blob/main/scripts/ncbi_gene_results_to_python.py)which generates genes_ncbi_human_proteincoding.py

In [ ]:
!ncbi_gene_results_to_python.py -o genes_ncbi_human_proteincoding.py $input_2

In [ ]:
# This step can not be performed earlier
from genes_ncbi_human_proteincoding import GENEID2NT as GeneID2nt_hum

In [ ]:
# !wget -c ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
!gunzip -f gene2go.gz
obo_fname = download_go_basic_obo()
fin_gene2go = download_ncbi_associations()
obodag = GODag("go-basic.obo")

In [ ]:
mapper = {}

for key in GeneID2nt_hum:
    mapper[GeneID2nt_hum[key].Symbol] = GeneID2nt_hum[key].GeneID
    
inv_map = {v: k for k, v in mapper.items()}

In [ ]:
len(inv_map)

In [ ]:
# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[10090]) # 9606 is for humans
# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

In [ ]:
goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_hum.keys(), # List of protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method
GO_items = {}

grouping = {}
grouping["BP"] = {}
GO_items["BP"] = []
temp = goeaobj.ns2objgoea['BP'].assoc
for item in temp:
    GO_items["BP"] += temp[item]
    for x in temp[item]:
        grouping["BP"][x] = item
    


grouping["CC"] = {}
GO_items["CC"] = []
temp = goeaobj.ns2objgoea['CC'].assoc
for item in temp:
    GO_items["CC"] += temp[item]
    for x in temp[item]:
        grouping["CC"][x] = item

grouping["MF"] = {}
GO_items["MF"] = []
temp = goeaobj.ns2objgoea['MF'].assoc
for item in temp:
    GO_items["MF"] += temp[item]
    for x in temp[item]:
        grouping["MF"][x] = item





#pass list of gene symbols
def go_it(test_genes, GO_items):
    print(f'input genes: {len(test_genes)}')
    
    mapped_genes = []
    for gene in test_genes:
        try:
            mapped_genes.append(mapper[gene])
        except:
            pass
    print(f'mapped genes: {len(mapped_genes)}')

    
    goea_results_all = goeaobj.run_study(mapped_genes)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    GO = pd.DataFrame(list(map(lambda x: [x.GO, x.goterm.name, x.goterm.namespace, x.p_uncorrected, x.p_fdr_bh,\
                   x.ratio_in_study[0], x.ratio_in_study[1], GO_items.count(x.GO), list(map(lambda y: inv_map[y], x.study_items)),\
                   ], goea_results_sig)), columns = ['GO', 'term', 'class', 'p', 'p_corr', 'n_genes',\
                                                    'n_study', 'n_go', 'study_genes'])

    GO = GO[GO.n_genes > 1]
    return GO



In [ ]:
# grouping

## Upregulated significant genes


In [ ]:
data1_u_s = data1[(data1["P.Value"]<0.05)&(data1["Log FC"]<0.0)]
genes = data1_u_s.Gene.values



In [ ]:
len(data1)

In [ ]:
# GO_items["BP"]

### BP

In [ ]:
u_s_df_fixed_BP = go_it(genes, GO_items["BP"])
# u_s_df_fixed_BP = u_s_df_fixed.copy()
u_s_df_fixed_BP['per'] = u_s_df_fixed_BP.n_genes/u_s_df_fixed_BP.n_go
# u_s_df_fixed_BP = u_s_df_fixed_BP[u_s_df_fixed_BP["per"] != np.inf]

In [ ]:
cmap = mpl.cm.bwr_r
norm = mpl.colors.Normalize(vmin = u_s_df_fixed_BP.p_corr.min(), vmax = u_s_df_fixed_BP.p_corr.max())

mappr = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)
sm = plt.cm.ScalarMappable(cmap=cm.bwr_r, norm=norm)
sm.set_array([])
ax = sns.barplot(data = u_s_df_fixed_BP, x = 'per', y = 'term', palette = mappr.to_rgba(u_s_df_fixed_BP.p_corr.values))
ax.figure.colorbar(sm)
ylim(-1, len(u_s_df_fixed_BP))
savefig("barplot_BP_upregulated.png")

In [ ]:
# NOTE: if per is inf, only name will appear in plot

In [ ]:
u_s_df_fixed_BP

In [ ]:
ontology_group = {}

for _, row in u_s_df_fixed_BP[["GO", "study_genes"]].iterrows():
    for x in row["study_genes"]:
        if x in ontology_group:
            ontology_group[x].append(row["GO"])
        else:
            ontology_group[x] = [row["GO"]]
            
            
# ontololu abc
ontology_group_type_n_id = {}

for x in ontology_group:
    ontology_group_type_n_id[x] = {}
    for go in ontology_group[x]:
        for gp in ["BP"]:
            ontology_group_type_n_id[x][gp] = []
            if go in grouping[gp]:
                ontology_group_type_n_id[x][gp].append(grouping[gp][go])
                
                
for gene in ontology_group_type_n_id:
    for tp in ontology_group_type_n_id[gene]:
        u_s_df_fixed_BP.loc[data1["Gene"]==gene, tp] = ",".join(map(str,ontology_group_type_n_id[gene][tp]))

u_s_df_fixed_BP.to_csv("Final_table_BP_upregulated.csv", index=False)

In [ ]:
# u_s_df_fixed_MF

### MF

In [ ]:
u_s_df_fixed_MF = go_it(genes, GO_items["MF"])
# u_s_df_fixed_BP = u_s_df_fixed.copy()
u_s_df_fixed_MF['per'] = u_s_df_fixed_MF.n_genes/u_s_df_fixed_MF.n_go
# u_s_df_fixed_BP = u_s_df_fixed_BP[u_s_df_fixed_BP["per"] != np.inf]

cmap = mpl.cm.bwr_r
norm = mpl.colors.Normalize(vmin = u_s_df_fixed_MF.p_corr.min(), vmax = u_s_df_fixed_MF.p_corr.max())

mappr = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)
sm = plt.cm.ScalarMappable(cmap=cm.bwr_r, norm=norm)
sm.set_array([])
ax = sns.barplot(data = u_s_df_fixed_MF, x = 'per', y = 'term', palette = mappr.to_rgba(u_s_df_fixed_MF.p_corr.values))
ax.figure.colorbar(sm)
ylim(-1, len(u_s_df_fixed_MF))
savefig("barplot_MF_upregulated.png")



In [ ]:
ontology_group = {}

for _, row in u_s_df_fixed_MF[["GO", "study_genes"]].iterrows():
    for x in row["study_genes"]:
        if x in ontology_group:
            ontology_group[x].append(row["GO"])
        else:
            ontology_group[x] = [row["GO"]]
            
            
# ontololu abc
ontology_group_type_n_id = {}

for x in ontology_group:
    ontology_group_type_n_id[x] = {}
    for go in ontology_group[x]:
        for gp in ["MF"]:
            ontology_group_type_n_id[x][gp] = []
            if go in grouping[gp]:
                ontology_group_type_n_id[x][gp].append(grouping[gp][go])
                
                
for gene in ontology_group_type_n_id:
    for tp in ontology_group_type_n_id[gene]:
        u_s_df_fixed_MF.loc[data1["Gene"]==gene, tp] = ",".join(map(str,ontology_group_type_n_id[gene][tp]))

u_s_df_fixed_MF.to_csv("Final_table_MF_upregulated.csv", index=False)

### CC

In [ ]:
u_s_df_fixed_CC = go_it(genes, GO_items["CC"])
# u_s_df_fixed_BP = u_s_df_fixed.copy()
u_s_df_fixed_CC['per'] = u_s_df_fixed_CC.n_genes/u_s_df_fixed_CC.n_go
# u_s_df_fixed_BP = u_s_df_fixed_BP[u_s_df_fixed_BP["per"] != np.inf]

cmap = mpl.cm.bwr_r
norm = mpl.colors.Normalize(vmin = u_s_df_fixed_CC.p_corr.min(), vmax = u_s_df_fixed_CC.p_corr.max())

mappr = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)
sm = plt.cm.ScalarMappable(cmap=cm.bwr_r, norm=norm)
sm.set_array([])
ax = sns.barplot(data = u_s_df_fixed_CC, x = 'per', y = 'term', palette = mappr.to_rgba(u_s_df_fixed_CC.p_corr.values))
ax.figure.colorbar(sm)
ylim(-1, len(u_s_df_fixed_CC))
savefig("barplot_MF_upregulated.png")

In [ ]:
ontology_group = {}

for _, row in u_s_df_fixed_CC[["GO", "study_genes"]].iterrows():
    for x in row["study_genes"]:
        if x in ontology_group:
            ontology_group[x].append(row["GO"])
        else:
            ontology_group[x] = [row["GO"]]
            
            
# ontololu abc
ontology_group_type_n_id = {}

for x in ontology_group:
    ontology_group_type_n_id[x] = {}
    for go in ontology_group[x]:
        for gp in ["MF"]:
            ontology_group_type_n_id[x][gp] = []
            if go in grouping[gp]:
                ontology_group_type_n_id[x][gp].append(grouping[gp][go])
                
                
for gene in ontology_group_type_n_id:
    for tp in ontology_group_type_n_id[gene]:
        u_s_df_fixed_CC.loc[data1["Gene"]==gene, tp] = ",".join(map(str,ontology_group_type_n_id[gene][tp]))

u_s_df_fixed_CC.to_csv("Final_table_CC_upregulated.csv", index=False)

### Down regulated significant genes

In [ ]:
data1_d_s = data1[(data1["P.Value"]<0.05)&(data1["Log FC"]>0.0)]
genes = data1_d_s.Gene.values


### BP

In [ ]:
d_s_df_fixed_BP = go_it(genes,  GO_items["BP"])
d_s_df_fixed_BP['per'] = d_s_df_fixed_BP.n_genes/d_s_df_fixed_BP.n_go


cmap = mpl.cm.bwr_r
norm = mpl.colors.Normalize(vmin = d_s_df_fixed_BP.p_corr.min(), vmax = d_s_df_fixed_BP.p_corr.max())

mappr = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)
sm = plt.cm.ScalarMappable(cmap=cm.bwr_r, norm=norm)
sm.set_array([])
ax = sns.barplot(data = d_s_df_fixed_BP, x = 'per', y = 'term', palette = mappr.to_rgba(d_s_df_fixed_BP.p_corr.values))
ax.figure.colorbar(sm)
ylim(-1, len(d_s_df_fixed_BP))
savefig("barplot_BP_downregulated.png")


In [ ]:
ontology_group = {}

for _, row in d_s_df_fixed_BP[["GO", "study_genes"]].iterrows():
    for x in row["study_genes"]:
        if x in ontology_group:
            ontology_group[x].append(row["GO"])
        else:
            ontology_group[x] = [row["GO"]]
            
            
# ontololu abc
ontology_group_type_n_id = {}

for x in ontology_group:
    ontology_group_type_n_id[x] = {}
    for go in ontology_group[x]:
        for gp in ["BP"]:
            ontology_group_type_n_id[x][gp] = []
            if go in grouping[gp]:
                ontology_group_type_n_id[x][gp].append(grouping[gp][go])
                
                
for gene in ontology_group_type_n_id:
    for tp in ontology_group_type_n_id[gene]:
        d_s_df_fixed_BP.loc[data1["Gene"]==gene, tp] = ",".join(map(str,ontology_group_type_n_id[gene][tp]))

d_s_df_fixed_BP.to_csv("Final_table_BP_downregulated.csv", index=False)

### MF

In [ ]:
d_s_df_fixed_MF = go_it(genes,  GO_items["MF"])
d_s_df_fixed_MF['per'] = d_s_df_fixed_MF.n_genes/d_s_df_fixed_MF.n_go


cmap = mpl.cm.bwr_r
norm = mpl.colors.Normalize(vmin = d_s_df_fixed_MF.p_corr.min(), vmax = d_s_df_fixed_MF.p_corr.max())

mappr = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)
sm = plt.cm.ScalarMappable(cmap=cm.bwr_r, norm=norm)
sm.set_array([])
ax = sns.barplot(data = d_s_df_fixed_MF, x = 'per', y = 'term', palette = mappr.to_rgba(d_s_df_fixed_MF.p_corr.values))
ax.figure.colorbar(sm)
ylim(-1, len(d_s_df_fixed_MF))
savefig("barplot_MF_downregulated.png")


In [ ]:
ontology_group = {}

for _, row in d_s_df_fixed_MF[["GO", "study_genes"]].iterrows():
    for x in row["study_genes"]:
        if x in ontology_group:
            ontology_group[x].append(row["GO"])
        else:
            ontology_group[x] = [row["GO"]]
            
            
# ontololu abc
ontology_group_type_n_id = {}

for x in ontology_group:
    ontology_group_type_n_id[x] = {}
    for go in ontology_group[x]:
        for gp in ["MF"]:
            ontology_group_type_n_id[x][gp] = []
            if go in grouping[gp]:
                ontology_group_type_n_id[x][gp].append(grouping[gp][go])
                
                
for gene in ontology_group_type_n_id:
    for tp in ontology_group_type_n_id[gene]:
        d_s_df_fixed_MF.loc[data1["Gene"]==gene, tp] = ",".join(map(str,ontology_group_type_n_id[gene][tp]))

d_s_df_fixed_MF.to_csv("Final_table_MF_downregulated.csv", index=False)

### CC

In [ ]:
d_s_df_fixed_CC = go_it(genes,  GO_items["CC"])
d_s_df_fixed_CC['per'] = d_s_df_fixed_CC.n_genes/d_s_df_fixed_CC.n_go


cmap = mpl.cm.bwr_r
norm = mpl.colors.Normalize(vmin = d_s_df_fixed_CC.p_corr.min(), vmax = d_s_df_fixed_CC.p_corr.max())

mappr = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)
sm = plt.cm.ScalarMappable(cmap=cm.bwr_r, norm=norm)
sm.set_array([])
ax = sns.barplot(data = d_s_df_fixed_CC, x = 'per', y = 'term', palette = mappr.to_rgba(d_s_df_fixed_CC.p_corr.values))
ax.figure.colorbar(sm)
ylim(-1, len(d_s_df_fixed_CC))
savefig("barplot_CC_downregulated.png")


In [ ]:
ontology_group = {}

for _, row in d_s_df_fixed_CC[["GO", "study_genes"]].iterrows():
    for x in row["study_genes"]:
        if x in ontology_group:
            ontology_group[x].append(row["GO"])
        else:
            ontology_group[x] = [row["GO"]]
            
            
# ontololu abc
ontology_group_type_n_id = {}

for x in ontology_group:
    ontology_group_type_n_id[x] = {}
    for go in ontology_group[x]:
        for gp in ["CC"]:
            ontology_group_type_n_id[x][gp] = []
            if go in grouping[gp]:
                ontology_group_type_n_id[x][gp].append(grouping[gp][go])
                
                
for gene in ontology_group_type_n_id:
    for tp in ontology_group_type_n_id[gene]:
        d_s_df_fixed_CC.loc[data1["Gene"]==gene, tp] = ",".join(map(str,ontology_group_type_n_id[gene][tp]))

d_s_df_fixed_CC.to_csv("Final_table_MF_downregulated.csv", index=False)